In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv(r'C:\Users\Mohamed Fawzi\Desktop\RFM Analysis\rfm_data.csv')
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


### Preparing Data

In [3]:
#convert CustomerID to string
df['CustomerID'] = df['CustomerID'].astype(str)

#convert PurchaseDate to datetime
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'])

#check for null values
df.isnull().sum()

#check for duplicates
df.drop_duplicates(inplace=True)

df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [4]:
df.describe()

,PurchaseDate,TransactionAmount,OrderID
count,1000,1000.00000,1000.000000
mean,2023-05-10 07:27:50.400000,513.67781,554071.398000
min,2023-04-11 00:00:00,12.13000,100096.000000
25%,2023-04-26 00:00:00,257.12000,313152.000000
50%,2023-05-09 00:00:00,523.56500,564671.500000
75%,2023-05-26 00:00:00,759.86000,783052.250000
max,2023-06-10 00:00:00,999.44000,999695.000000
std,NaN,286.09870,264695.448814
